## Дана таблица с звонками на горячую линию. Необходимо рассчитать сколько необходимо операторов на линии каждые полчаса, чтобы 80% звонков были приняты в первые 30 секунд. Используется калькулятор Erlang

In [1]:
from math import factorial, ceil, exp
import pandas as pd
import numpy as np
import datetime as dt

In [2]:
def ErlangC(A, N):
    if (N-A<=0): 
        return 1 
    L = (A**N / factorial(N)) * (N / (N - A))
    sum_ = 0
    for i in range(N):
        sum_ += (A**i) / factorial(i)
    return (L / (sum_ + L))


In [3]:
data = pd.read_excel("C:/Users/HP/Downloads/95043f7f9fbffe1b00e9469ff2413732.xlsx", parse_dates=["A_CallDTBegin"])

In [4]:
data["time_period"] = pd.to_datetime(data.A_CallDTBegin.dt.hour.astype(str) + ":" + data.A_CallDTBegin.dt.minute.apply(lambda x: "30" if x>29 else "00")).dt.strftime("%H:%M")

In [5]:
countable_calls = data[data["Общая Длительность звонка в секундах с момента попадания в IVR"] > 26]


In [6]:
erl_table = countable_calls.groupby("time_period", as_index=False).agg({"ID" : "count"}).rename(columns={"ID" : "calls"})

In [7]:
aht = countable_calls[countable_calls["Длительность диалога с оператором в секундах"] > 0]["Длительность диалога с оператором в секундах"].mean()
aht

172.9686526479751

In [8]:
erl_table["calls_per_hour"] = erl_table.calls * 2

In [9]:
erl_table["A"] = erl_table.calls_per_hour * aht / 3600

In [10]:
erl_table.head()

,time_period,calls,calls_per_hour,A
0,00:00,20,40,1.921874
1,00:30,6,12,0.576562
2,01:00,14,28,1.345312
3,01:30,12,24,1.153124
4,02:00,12,24,1.153124


In [11]:
def calc_sl(A, aht, tt, perc):
    for staff in range(ceil(A), 1000):
        P = ErlangC(A, staff)
        sl = (1 - (P * exp(-((staff - A) * (tt / aht))))) * 100
        if sl >= perc: # percent of calls to be in tagreted waiting time
            return staff
    

In [12]:
list1 =[]
for i in range(erl_table.shape[0]):
    list1.append(calc_sl(erl_table["A"][i], aht, 30, 80))


In [13]:
erl_table["staff"] = list1

In [14]:
erl_table.head()

,time_period,calls,calls_per_hour,A,staff
0,00:00,20,40,1.921874,4
1,00:30,6,12,0.576562,2
2,01:00,14,28,1.345312,3
3,01:30,12,24,1.153124,3
4,02:00,12,24,1.153124,3


In [162]:
erl_table.to_excel("C:/Users/HP/Downloads/Erlang.xlsx")